## Imports

In [3]:
# general imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from convokit import Corpus, download
from tqdm import tqdm
import nltk

# syntactic specific imports
from nltk import pos_tag
from nltk.corpus import treebank
from nltk.tree import *
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
nltk.download('treebank')

import spacy
nlp = spacy.load("en_core_web_sm") # pre-trained English model

import stanza
stanza.download("en")
stanza_parser = stanza.Pipeline("en", processors="tokenize,pos,constituency")

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/nickvick/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
2026-01-26 18:27:10 INFO: Downloaded file to /Users/nickvick/stanza_resources/resources.json
2026-01-26 18:27:10 INFO: Downloading default packages for language: en (English) ...
2026-01-26 18:27:11 INFO: File exists: /Users/nickvick/stanza_resources/en/default.zip
2026-01-26 18:27:12 INFO: Finished downloading models and saved to /Users/nickvick/stanza_resources
2026-01-26 18:27:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2026-01-26 18:27:12 INFO: Downloaded file to /Users/nickvick/stanza_resources/resources.json
2026-01-26 18:27:12 WARNING: Language en package default expects mwt, which has been added
2026-01-26 18:27:12 INFO: Loading these models for language: en (English):
| Processor    | P

In [4]:
# set up for src imports
import sys
import os

# add project root to sys.path (so src/ can be imported)
project_root = os.path.abspath("..")  # adjust if notebooks are nested deeper
if project_root not in sys.path:
    sys.path.append(project_root)

# import required functions
from src.data_preprocessing import corpus_to_df, syntactic_preprocessing_df, is_complete_sentence

## Syntactic Analysis Functions

## Data Analysis

## Visualizations